<h1>Part 4 - Experiment Tracking</h1>

# Experiment Tracking and Model Management with MLFlow

There are many ways to use the MLFlow Tracking API. For simple local uses, the best is to leave the data management to MLFlow and let it store runs, metrics, models and artifacts locally. For more advanced usage, all of this information can be stored in databases. You can find the detailed on MLFlow's documentation [here](https://mlflow.org/docs/latest/tracking.html#scenario-1-mlflow-on-localhost).

## Exploring MLFlow

MLflow setup:
* Tracking server: no
* Backend store: local filesystem
* Artifacts store: local filesystem

The experiments can be explored locally by launching the MLflow UI.

Let's print the tracking server URI, where the experiments and runs are going to be logged. We observe it refers to a local path.

In [1]:
import mlflow

print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'file:///d:/Vinay/ESILV/machine-learning-operations/esilv-mlops-crashcourse/lessons/01-model-and-experiment-management/mlruns'


c:\Users\vinay\anaconda3\envs\mlops-course\lib\site-packages\pydantic\_internal\_fields.py:128: UserWarning: Field "model_server_url" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
c:\Users\vinay\anaconda3\envs\mlops-course\lib\site-packages\pydantic\_internal\_config.py:317: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)


After this initialization, we can connect create a client to connect to the API and see what experiments are present.

By refering to mlflow's [documentation](https://mlflow.org/docs/latest/python_api/mlflow.client.html), create a client and display a list of the available experiments using the search_experiments function. This function could prove useful later to programatically explore experiments (rather than in the UI)

In [2]:
from mlflow.tracking import MlflowClient

import mlflow
from mlflow.tracking import MlflowClient

# Create an MLflow client
client = MlflowClient()

# Search for experiments
experiments = client.search_experiments()
experiments 
# Display the list of available experiments
# print("Available Experiments:")
# for experiment in experiments:
#     print(f"- Name: {experiment.name}")
#     print(f"  ID: {experiment.experiment_id}")
#     print(f"  Artifact Location: {experiment.artifact_location}")
#     print(f"  Lifecycle Stage: {experiment.lifecycle_stage}")
#     print(f"  Creation Time: {experiment.creation_time}")
#     print(f"  Last Update Time: {experiment.last_update_time}")
#     print("---")

[<Experiment: artifact_location='file:///d:/Vinay/ESILV/machine-learning-operations/esilv-mlops-crashcourse/lessons/01-model-and-experiment-management/mlruns/503983593208481221', creation_time=1736108651989, experiment_id='503983593208481221', last_update_time=1736108651989, lifecycle_stage='active', name='/mlflow/xgboost_test', tags={}>,
 <Experiment: artifact_location='file:///d:/Vinay/ESILV/machine-learning-operations/esilv-mlops-crashcourse/lessons/01-model-and-experiment-management/mlruns/683247729893304907', creation_time=1736108046508, experiment_id='683247729893304907', last_update_time=1736108046508, lifecycle_stage='active', name='/mlflow/linear_reg_test', tags={}>,
 <Experiment: artifact_location='file:///d:/Vinay/ESILV/ML/esilv-mlops-crashcourse/lessons/01-model-and-experiment-management/mlruns/200577488369321930', creation_time=1729495692089, experiment_id='200577488369321930', last_update_time=1729495692089, lifecycle_stage='active', name='taxi-duration-prediction', tags=

We see that there is a default experiment for which the runs are stored locally in the mlruns folder.

### Creating an experiment and logging a new run

An experiment is a logical entity regrouping the logs of multiple attempts at solving a same problem, called runs. \
We will now work with the classic sklearn dataset iris. Our goal here is to manage to classify the different iris species. To track our models performance, we will log every attempt as a "run" and create a new experiment "iris-experiment-1" to regroup them.

Lookup the mlflow.run and mlflow.start_run functions [here](https://mlflow.org/docs/latest/python_api/mlflow.html?highlight=start_run#mlflow.start_run) to find out how to manage runs.
Explore [this part](https://mlflow.org/docs/latest/python_api/mlflow.html) to learn more about the log_params, log_metrics and log_artifact functions. Find out how to log sklearn models [here](https://mlflow.org/docs/latest/python_api/mlflow.sklearn.html])

Complete the following in order to log the parameters, interesting metrics and the model.

In [3]:
import mlflow
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split

mlflow.set_experiment("iris-experiment-1")

with mlflow.start_run() as run:
    run_id = run.info.run_id

    # Load the Iris dataset
    X, y = load_iris(return_X_y=True)

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Set and log parameters
    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    # Train the model
    model = LogisticRegression(**params).fit(X_train, y_train)

    # Make predictions
    y_pred = model.predict(X_test)

    # Calculate and log metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    mlflow.log_metrics({
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1
    })

    # Log the model
    mlflow.sklearn.log_model(model, artifact_path="model")

    # print(f"Run ID: {run_id}")
    # print(f"Accuracy: {accuracy:.4f}")
    # print(f"Precision: {precision:.4f}")
    # print(f"Recall: {recall:.4f}")
    # print(f"F1 Score: {f1:.4f}")
    print(f"Default artifacts URI: '{mlflow.get_artifact_uri()}'")

Default artifacts URI: 'file:///d:/Vinay/ESILV/ML/esilv-mlops-crashcourse/lessons/01-model-and-experiment-management/mlruns/618500271556947388/639d13771bc2446bb1cef967bfb8b493/artifacts'


In [4]:
experiments = client.search_experiments()
experiments

[<Experiment: artifact_location='file:///d:/Vinay/ESILV/machine-learning-operations/esilv-mlops-crashcourse/lessons/01-model-and-experiment-management/mlruns/503983593208481221', creation_time=1736108651989, experiment_id='503983593208481221', last_update_time=1736108651989, lifecycle_stage='active', name='/mlflow/xgboost_test', tags={}>,
 <Experiment: artifact_location='file:///d:/Vinay/ESILV/machine-learning-operations/esilv-mlops-crashcourse/lessons/01-model-and-experiment-management/mlruns/683247729893304907', creation_time=1736108046508, experiment_id='683247729893304907', last_update_time=1736108046508, lifecycle_stage='active', name='/mlflow/linear_reg_test', tags={}>,
 <Experiment: artifact_location='file:///d:/Vinay/ESILV/ML/esilv-mlops-crashcourse/lessons/01-model-and-experiment-management/mlruns/200577488369321930', creation_time=1729495692089, experiment_id='200577488369321930', last_update_time=1729495692089, lifecycle_stage='active', name='taxi-duration-prediction', tags=

Try running the training script with various parameters to have runs to compare.
You can now explore your run(s) using the ui: \
(Paste "mlflow ui --host 0.0.0.0 --port 5002" in your terminal, or run the cell below)

**N.B.** Make sure you are in the lecture folder and not the repo root!

In [5]:
!mlflow ui --host 0.0.0.0 --port 5002

^C


You will have to kill the cell to continue experimenting

### Interacting with the model registry

If you are satisfied with the last run's model, you can transform the logged model into a registered model. It will be logged in the Model Registry, which makes it easier to use in production and manage versions.

In [6]:
# We already have our run id from above. Let's use it to register the model

result = mlflow.register_model(f"runs:/{run_id}/models", "iris_lr_model")

Registered model 'iris_lr_model' already exists. Creating a new version of this model...
2025/01/05 22:24:08 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: iris_lr_model, version 21
Created version '21' of model 'iris_lr_model'.


# Use Case

Now we will get back to our taxi rides use case: 

In [7]:
import pandas as pd
import seaborn as sns
import numpy as np

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error

from typing import List
from scipy.sparse import csr_matrix

## 0 - Download Data

In [8]:
!pip install gdown

In [9]:
import gdown
import os

DATA_FOLDER = "../../data"
train_path = f"{DATA_FOLDER}/yellow_tripdata_2021-01.parquet"
test_path = f"{DATA_FOLDER}/yellow_tripdata_2021-02.parquet"
predict_path = f"{DATA_FOLDER}/yellow_tripdata_2021-03.parquet"


if not os.path.exists(DATA_FOLDER):
    os.makedirs(DATA_FOLDER)
    print(f"New directory {DATA_FOLDER} created!")

gdown.download(
    "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2021-01.parquet",
    train_path,
    quiet=False,
)
gdown.download(
    "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2021-02.parquet",
    test_path,
    quiet=False,
)
gdown.download(
    "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2021-03.parquet",
    predict_path,
    quiet=False,
)

Downloading...
From: https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2021-01.parquet
To: d:\Vinay\ESILV\machine-learning-operations\esilv-mlops-crashcourse\data\yellow_tripdata_2021-01.parquet
100%|██████████| 21.7M/21.7M [00:25<00:00, 840kB/s] 
Downloading...
From: https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2021-02.parquet
To: d:\Vinay\ESILV\machine-learning-operations\esilv-mlops-crashcourse\data\yellow_tripdata_2021-02.parquet
100%|██████████| 21.8M/21.8M [00:17<00:00, 1.23MB/s]
Downloading...
From: https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2021-03.parquet
To: d:\Vinay\ESILV\machine-learning-operations\esilv-mlops-crashcourse\data\yellow_tripdata_2021-03.parquet
100%|██████████| 30.0M/30.0M [00:23<00:00, 1.27MB/s]


'../../data/yellow_tripdata_2021-03.parquet'

## 1 - Load data

In [10]:
def load_data(path: str):
    return pd.read_parquet(path)


train_df = load_data(train_path)
train_df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2021-01-01 00:30:10,2021-01-01 00:36:12,1.0,2.10,1.0,N,142,43,2,8.0,3.0,0.5,0.00,0.0,0.3,11.80,2.5,NaN
1,1,2021-01-01 00:51:20,2021-01-01 00:52:19,1.0,0.20,1.0,N,238,151,2,3.0,0.5,0.5,0.00,0.0,0.3,4.30,0.0,NaN
2,1,2021-01-01 00:43:30,2021-01-01 01:11:06,1.0,14.70,1.0,N,132,165,1,42.0,0.5,0.5,8.65,0.0,0.3,51.95,0.0,NaN
3,1,2021-01-01 00:15:48,2021-01-01 00:31:01,0.0,10.60,1.0,N,138,132,1,29.0,0.5,0.5,6.05,0.0,0.3,36.35,0.0,NaN
4,2,2021-01-01 00:31:49,2021-01-01 00:48:21,1.0,4.94,1.0,N,68,33,1,16.5,0.5,0.5,4.06,0.0,0.3,24.36,2.5,NaN


## 2 - Prepare the data

Let's prepare the data to make it Machine Learning ready. \
For this, we need to clean it, compute the target (what we want to predict), and compute some features to help the model understand the data better.

### 2-1 Compute the target

We want to predict a taxi trip duration in minutes. Let's compute it as a difference between the drop-off time and the pick-up time for each trip.

In [11]:
def compute_target(
    df: pd.DataFrame,
    pickup_column: str = "tpep_pickup_datetime",
    dropoff_column: str = "tpep_dropoff_datetime",
) -> pd.DataFrame:
    df["duration"] = df[dropoff_column] - df[pickup_column]
    df["duration"] = df["duration"].dt.total_seconds() / 60
    return df


train_df = compute_target(train_df)

In [12]:
train_df["duration"].describe()

count    1.369769e+06
mean     1.391168e+01
std      1.312006e+02
min     -1.350846e+05
25%      5.566667e+00
50%      9.066667e+00
75%      1.461667e+01
max      2.881770e+04
Name: duration, dtype: float64

Let's remove outliers and reduce the scope to trips between 1 minute and 1 hour

In [13]:
MIN_DURATION = 1
MAX_DURATION = 60


def filter_outliers(df: pd.DataFrame, min_duration: int = 1, max_duration: int = 60) -> pd.DataFrame:
    return df[df["duration"].between(min_duration, max_duration)]


train_df = filter_outliers(train_df)

### 2-2 Prepare features

#### 2-2-1 Categorical features

Most machine learning models don't work with categorical features. Because of this, they must be transformed so that the ML model can consume them.

In [14]:
CATEGORICAL_COLS = ["PUlocationID", "DOlocationID"]


def encode_categorical_cols(df: pd.DataFrame, categorical_cols: List[str] = None) -> pd.DataFrame:
    if categorical_cols is None:
        categorical_cols = ["PULocationID", "DOLocationID", "passenger_count"]
    df[categorical_cols] = df[categorical_cols].fillna(-1).astype("int")
    df[categorical_cols] = df[categorical_cols].astype("str")
    return df


train_df = encode_categorical_cols(train_df)

In [15]:
def extract_x_y(
    df: pd.DataFrame,
    categorical_cols: List[str] = None,
    dv: DictVectorizer = None,
    with_target: bool = True,
) -> dict:

    if categorical_cols is None:
        categorical_cols = ["PULocationID", "DOLocationID", "passenger_count"]
    dicts = df[categorical_cols].to_dict(orient="records")

    y = None
    if with_target:
        if dv is None:
            dv = DictVectorizer()
            dv.fit(dicts)
        y = df["duration"].values

    x = dv.transform(dicts)
    return x, y, dv


X_train, y_train, dv = extract_x_y(train_df)

## 3 - Train model

We train a basic linear regression model to have a baseline performance

In [16]:
def train_model(x_train: csr_matrix, y_train: np.ndarray):
    lr = LinearRegression()
    lr.fit(x_train, y_train)
    return lr


model = train_model(X_train, y_train)

## 4 - Evaluate model

We evaluate the model on train and test data

### 4-1 On train data

In [17]:
def predict_duration(input_data: csr_matrix, model: LinearRegression):
    return model.predict(input_data)


def evaluate_model(y_true: np.ndarray, y_pred: np.ndarray):
    return mean_squared_error(y_true, y_pred, squared=False)


prediction = predict_duration(X_train, model)
train_me = evaluate_model(y_train, prediction)
train_me

6.782411766110919

### 4-2 On test data

In [18]:
test_df = load_data(test_path)

In [19]:
test_df = compute_target(test_df)
test_df = encode_categorical_cols(test_df)
X_test, y_test, _ = extract_x_y(test_df, dv=dv)

In [20]:
y_pred_test = predict_duration(X_test, model)
test_me = evaluate_model(y_test, y_pred_test)
test_me

58.37505304309019

## 5 - Log Model Parameters to MlFlow

Now that all our development functions are built and tested, let's create a training pipeline and log the training parameters, logs and model to MlFlow.

Create a training flow, log all the important parameters, metrics and model. Try to find what could be important and needs to be logged.

In [21]:
import mlflow
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Set the experiment name
mlflow.set_experiment("taxi-duration-prediction")

# Start a run
with mlflow.start_run() as run:
    run_id = run.info.run_id

    # Set tags for the run
    mlflow.set_tag("developer", "YourName")
    mlflow.set_tag("dataset", "NYC Yellow Taxi 2021")

    # Load data
    train_df = load_data(train_path)
    test_df = load_data(test_path)

    # Log data info
    mlflow.log_param("train_data_path", train_path)
    mlflow.log_param("test_data_path", test_path)
    mlflow.log_param("train_data_rows", len(train_df))
    mlflow.log_param("test_data_rows", len(test_df))

    # Compute target
    train_df = compute_target(train_df)
    test_df = compute_target(test_df)

    # Filter outliers
    MIN_DURATION = 1
    MAX_DURATION = 60
    train_df = filter_outliers(train_df, MIN_DURATION, MAX_DURATION)
    test_df = filter_outliers(test_df, MIN_DURATION, MAX_DURATION)

    mlflow.log_param("min_duration", MIN_DURATION)
    mlflow.log_param("max_duration", MAX_DURATION)
    mlflow.log_param("filtered_train_data_rows", len(train_df))
    mlflow.log_param("filtered_test_data_rows", len(test_df))

    # Encode categorical columns
    # Check for the correct column names
    possible_pu_columns = ["PULocationID", "PUlocationID", "pulocationid"]
    possible_do_columns = ["DOLocationID", "DOlocationID", "dolocationid"]

    pu_column = next((col for col in possible_pu_columns if col in train_df.columns), None)
    do_column = next((col for col in possible_do_columns if col in train_df.columns), None)

    if pu_column is None or do_column is None:
        raise ValueError(f"Required location columns not found. Available columns: {train_df.columns}")

    CATEGORICAL_COLS = [pu_column, do_column]

    def encode_categorical_cols(df, categorical_cols):
        df[categorical_cols] = df[categorical_cols].fillna(-1).astype("int")
        df[categorical_cols] = df[categorical_cols].astype("str")
        return df

    train_df = encode_categorical_cols(train_df, CATEGORICAL_COLS)
    test_df = encode_categorical_cols(test_df, CATEGORICAL_COLS)

    mlflow.log_param("categorical_columns", CATEGORICAL_COLS)

    # Extract X and y
    X_train, y_train, dv = extract_x_y(train_df, CATEGORICAL_COLS)
    X_test, y_test, _ = extract_x_y(test_df, CATEGORICAL_COLS, dv)

    # Train model
    model = train_model(X_train, y_train)

    # Log model parameters
    mlflow.log_params(model.get_params())

    # Evaluate model on train set
    y_pred_train = predict_duration(X_train, model)
    train_rmse = evaluate_model(y_train, y_pred_train)
    mlflow.log_metric("train_rmse", train_rmse)

    # Evaluate model on test set
    y_pred_test = predict_duration(X_test, model)
    test_rmse = evaluate_model(y_test, y_pred_test)
    mlflow.log_metric("test_rmse", test_rmse)

    # Log your model
    mlflow.sklearn.log_model(model, "taxi_duration_model")

    # Log feature names
    # Use get_feature_names_out() for newer scikit-learn versions
    # try:
    #     feature_names = dv.get_feature_names_out()
    # except AttributeError:
    #     # Fallback for older versions
    #     feature_names = dv.get_feature_names()
    
    # mlflow.log_param("feature_names", feature_names.tolist())

    print(f"Run ID: {run_id}")
    print(f"Train RMSE: {train_rmse}")
    print(f"Test RMSE: {test_rmse}")

    # Register your model in MLflow model registry
    model_uri = f"runs:/{run_id}/taxi_duration_model"
    registered_model = mlflow.register_model(model_uri, "taxi_duration_predictor")
    print(f"Model registered as: {registered_model.name}")

c:\Users\vinay\anaconda3\envs\mlops-course\lib\site-packages\_distutils_hack\__init__.py:16: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
c:\Users\vinay\anaconda3\envs\mlops-course\lib\site-packages\_distutils_hack\__init__.py:31: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
Registered model 'taxi_duration_predictor' already exists. Creating a new version of this model...
2025/01/05 22:27:02 IN

Run ID: e4fa34405fa141b2916136723e39c896
Train RMSE: 6.845620175515615
Test RMSE: 7.423728742609017
Model registered as: taxi_duration_predictor


Created version '6' of model 'taxi_duration_predictor'.


If the model is satisfactory, we stage it as production using the appropriate version. This will help us retreiving it for predictions.

Create a mlflow client and use the [mlflow documentation](https://mlflow.org/docs/latest/python_api/mlflow.client.html?highlight=transition_model_version_stage#mlflow.client.MlflowClient.transition_model_version_stage) to stage the appropriate model as being in "production".

In [22]:
from mlflow.tracking import MlflowClient
import mlflow

# Create an MLflow client
client = MlflowClient()

# Set the model name
model_name = "taxi_duration_predictor"

# Get all versions of the model
model_versions = client.search_model_versions(f"name='{model_name}'")

if not model_versions:
    print(f"No versions found for model '{model_name}'")
else:
    # Sort versions by creation timestamp (newest first)
    sorted_versions = sorted(model_versions, key=lambda x: x.creation_timestamp, reverse=True)
    
    # Get the latest version
    latest_version = sorted_versions[0]
    
    # Transition the latest version to "Production" stage
    client.transition_model_version_stage(
        name=model_name,
        version=latest_version.version,
        stage="Production"
    )
    
    print(f"Model '{model_name}' version {latest_version.version} transitioned to Production stage")
    
    # Optional: Archive other versions
    for version in sorted_versions[1:]:
        client.transition_model_version_stage(
            name=model_name,
            version=version.version,
            stage="Archived"
        )
        print(f"Model '{model_name}' version {version.version} archived")

# Verify the current production model
production_model = client.get_latest_versions(model_name, stages=["Production"])
if production_model:
    print(f"Current production model: {model_name} version {production_model[0].version}")
else:
    print("No production model found")

Model 'taxi_duration_predictor' version 6 transitioned to Production stage
Model 'taxi_duration_predictor' version 5 archived
Model 'taxi_duration_predictor' version 4 archived
Model 'taxi_duration_predictor' version 3 archived
Model 'taxi_duration_predictor' version 2 archived
Model 'taxi_duration_predictor' version 1 archived
Current production model: taxi_duration_predictor version 6


## 6 - Predict

We can now use our model to predict on fresh unseen data and forecast what is going to be the duration of a tawi trip depending on trip characteristics.

In [23]:
# Load prediction data
predict_df = load_data(predict_path)

# Apply feature engineering
CATEGORICAL_COLS = ["PULocationID", "DOLocationID"]  # Make sure these column names match your dataset
predict_df = encode_categorical_cols(predict_df, CATEGORICAL_COLS)
X_pred, _, _ = extract_x_y(predict_df, categorical_cols=CATEGORICAL_COLS, dv=dv, with_target=False)

# Load production model
model_name = "taxi_duration_predictor"
production_model = mlflow.sklearn.load_model(f"models:/{model_name}/production")

# Make predictions
y_pred = predict_duration(X_pred, production_model)
print(y_pred[:10])  # Print first 10 predictions

[11.36667603 11.92509575 11.92509575 26.69724174 10.3750759   9.33442362
 25.42918422 32.71698188 14.74052174 15.2367682 ]


## 7 - To go further

If you managed to go this far, you can try solving the use case using an other regression model like [XGBoost](https://xgboost.readthedocs.io/en/stable/) for instance.

In [24]:
############################################
from typing import Any
import pickle

def load_pickle(path: str):
    with open(path, "rb") as f:
        loaded_obj = pickle.load(f)
    return loaded_obj


def save_pickle(path: str, obj: Any):
    with open(path, "wb") as f:
        pickle.dump(obj, f)

save_pickle("dv__v0.0.5.pkl", dv)
save_pickle("model__v0.0.5.pkl", model)

############################################